In [ ]:
import pandas as pd
import numpy as np
import lightgbm
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
from pathlib import Path

In [ ]:
# Better rendering
from IPython.core.display import HTML
HTML("<style>.rendered_html th {max-width: 120px;}</style>")

warnings.filterwarnings('ignore')

# settings to display all columns
pd.set_option("display.max_columns", None)

In [ ]:
# Import data
# Specify the path to your CSV file
brand1_csv = "/Users/galuhprisillia/PycharmProjects/forecasting/brand1.csv"
brand2_csv = "/Users/galuhprisillia/PycharmProjects/forecasting/brand2.csv"
brand3_csv = "/Users/galuhprisillia/PycharmProjects/forecasting/brand3.csv"
brand4_csv = "/Users/galuhprisillia/PycharmProjects/forecasting/brand4.csv"
# Display the DataFrame

In [ ]:
# df_brand1 = pd.read_csv(brand1_csv)
df_brand1 = pd.read_csv(brand1_csv)
df_brand2 = pd.read_csv(brand2_csv)
df_brand3 = pd.read_csv(brand3_csv)
df_brand4 = pd.read_csv(brand4_csv)

df = pd.concat([df_brand1, df_brand2, df_brand3, df_brand4])

In [ ]:
df.head()

In [ ]:
# change date to datetime
df['date'] = pd.to_datetime(df['date'])

# rename default_code column to product_id
df = df.rename(columns={'default_code': 'product_id'})
# make a new column buy_quantity and sell_quantity
df['buy_quantity'] = df['product_qty'].apply(lambda x: x if x > 0 else 0)
df['sell_quantity'] = df['product_qty'].apply(lambda x: x if x < 0 else 0)

# make buy_quntity positive
df['buy_quantity'] = df['buy_quantity'].apply(lambda x: abs(x))
df['sell_quantity'] = df['sell_quantity'].apply(lambda x: abs(x))

# drop if is_pack is True
df = df[df.is_pack == False]

# drop reference column
df = df.drop(columns=['reference'])

# drop is_pack column
df = df.drop(columns=['is_pack'])

df = df.drop(columns=['product_qty'])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# sort the dataframe by product_id, and date and reset index
df_filter = df.sort_values(by=['product_id', 'date'])

df_filter = df_filter.reset_index(drop=True)

In [ ]:
df_filter.head()

In [ ]:
# make a new column total_quantity and make it cumulative sum
df_filter['stock_quantity'] = df_filter.groupby('product_id')['buy_quantity'].cumsum() - df_filter.groupby('product_id')['sell_quantity'].cumsum()

In [ ]:
df_filter.head(20)

In [ ]:
# check the min and max date
print(df_filter['date'].min())
print(df_filter['date'].max())

In [ ]:
item_A = df_filter[df_filter['product_id'] == '118471']
#item_A = df_filter[df_filter['product_id'] == 'SMB4G04']
item_B = df_filter[df_filter['product_id'] == '118464']
item_C = df_filter[df_filter['product_id'] == '118465']

In [ ]:
print('item_a',item_A['date'].min(), item_A['date'].max())
print('item_b',item_B['date'].min(), item_B['date'].max())
print('item_c',item_C['date'].min(), item_C['date'].max())

In [ ]:
# for now i wiill focus on item_A, the rest will follow the same process

In [ ]:
# Create a complete range of dates
complete_dates = pd.date_range(start=item_A['date'].min(), end=item_A['date'].max())

# Check if there are any skipped dates
skipped_dates = complete_dates[~complete_dates.isin(item_A['date'])]

if skipped_dates.empty:
    print("No skipped dates in the data")
else:
    print("Skipped dates in the data:", skipped_dates)

In [ ]:
# fill the skiped date using previos value
# Create a complete range of dates
complete_dates = pd.date_range(start=item_A['date'].min(), end=item_A['date'].max())

# Check if there are any skipped dates
skipped_dates = complete_dates[~complete_dates.isin(item_A['date'])]

if not skipped_dates.empty:
    # Create a DataFrame with the skipped dates and NaN values
    skipped_df = pd.DataFrame({'date': skipped_dates})
    skipped_df['product_id'] = np.nan
    skipped_df['list_price'] = np.nan
    skipped_df['buy_quantity'] = np.nan
    skipped_df['sell_quantity'] = np.nan
    skipped_df['stock_quantity'] = np.nan

    # Concatenate the original DataFrame and the skipped DataFrame
    item_A = pd.concat([item_A, skipped_df]).sort_values(by='date').reset_index(drop=True)

# Fill NaN values using previous value for specific columns
columns_to_fill = ['product_id', 'list_price', 'stock_quantity']
item_A[columns_to_fill] = item_A[columns_to_fill].fillna(method='ffill')

# Fill NaN values for 'buy_quantity' and 'sell_quantity' with 0
item_A[['buy_quantity', 'sell_quantity']] = item_A[['buy_quantity', 'sell_quantity']].fillna(0)

In [ ]:
# Create a complete range of dates
complete_dates = pd.date_range(start=item_A['date'].min(), end=item_A['date'].max())

# Check if there are any skipped dates
skipped_dates = complete_dates[~complete_dates.isin(item_A['date'])]

if skipped_dates.empty:
    print("No skipped dates in the data")
else:
    print("Skipped dates in the data:", skipped_dates)

In [ ]:
train = item_A.copy()

In [ ]:
train.head()

In [ ]:
# change the date into
train['date'] = train['date'].dt.date

In [ ]:
# drop list_price, buy_quantity, stock_quantity
# rename sell_quantity to sales
train = train.drop(columns=['buy_quantity'])
train = train.rename(columns={'sell_quantity': 'sales'})


In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
# grop by date and sum sales
train = train.groupby('date').agg({'sales': 'sum', 'stock_quantity': 'last' ,'list_price': 'mean'}).reset_index()

In [ ]:
train.head()

In [ ]:
import pandas as pd
import numpy as np

def date_features(dataset):
    # Date Features
    dataset['date'] = pd.to_datetime(dataset['date'])
    dataset['year'] = dataset['date'].dt.year
    dataset['month'] = dataset['date'].dt.month
    dataset['day'] = dataset['date'].dt.day
    dataset['dayofyear'] = dataset['date'].dt.dayofyear
    dataset['dayofweek'] = dataset['date'].dt.dayofweek
    dataset['weekofyear'] = dataset['date'].dt.isocalendar().week

    # Additional Data Features
    dataset['day^year'] = np.log((np.log(dataset['dayofyear'] + 1)) ** (dataset['year'] - 2000))

    return dataset

In [ ]:
# Dates Features for Train, Test
train = date_features(train)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
# Daily Average, Monthly Average for train
train['weekly_avg'] = train.groupby(pd.Grouper(key='date', freq='W'))['sales'].transform('mean')
train['monthly_avg'] = train.groupby(['year','month'])['sales'].transform('mean')
train = train.dropna()


In [ ]:
train.head()

In [ ]:
train.head()

In [ ]:
# rolling sold
train['rolling_sold_mean'] = train['sales'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)

for days in [3,7,14,21,28]:
    train['rolling_sold_mean_{}'.format(days)] = train['sales'].transform(lambda x: x.rolling(window=days).mean()).astype(np.float16)


In [ ]:
# rolling sold from
train['rolling_sold_mean'] = train['sales'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)

for days in [1,2,3,5,6,7,14,21,28]:
    train['rolling_sold_mean_{}'.format(days)] = train['sales'].transform(lambda x: x.rolling(window=days).mean()).astype(np.float16)

In [ ]:
# Introduce lags (days)
lags = [1, 2, 3, 4,5,6, 7, 14, 28]
for lag in lags:
    train['sold_lag_'+str(lag)] = train['sales'].shift(lag).astype(np.float16)

In [ ]:
train.head()

In [ ]:
# Rolling Average on actual lag
for window, lag in zip([7, 7, 28, 28], [7, 28, 7, 28]):
    train['rolling_lag_{}_win_{}'.format(window, lag)] = train['sold_lag_{}'.format(lag)].transform(lambda x: x.rolling(window=window).mean()).astype(np.float16)

In [ ]:
# Rolling Average on actual lag
for window, lag in zip([7, 7, 28, 28], [7, 28, 7, 28]):
    train['rolling_lag_{}_win_{}'.format(window, lag)] = train['sold_lag_{}'.format(lag)].transform(lambda x: x.rolling(window=window).mean()).astype(np.float16)

In [ ]:
# Average for the last n days
for days in [1, 2, 3, 5, 7, 14, 21, 28]:
    train['rolling_sold_weekly_avg_{}'.format(days)] = train['weekly_avg'].transform(lambda x: x.rolling(window=days).max()).astype(np.float16)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
fig, ax = plt.subplots(figsize=(50, 50))

# Create a heatmap of the correlation coefficients between the features
sns.heatmap(train.corr(), annot=True, fmt='.2f', ax=ax)

# Set the title and labels for the plot
ax.set_title('Correlation Heatmap')
ax.set_xlabel('Features')
ax.set_ylabel('Features')

In [ ]:
train.head()

In [ ]:
# Calculate the correlation between features and sales
correlation = train.corr()['sales'].sort_values(ascending=False)

# Print the top 10 features with the highest correlation to sales
top_features = correlation.head(7)
print(top_features)


In [ ]:
df_train = train[['date', 'sales', 'rolling_sold_mean_2', 'rolling_sold_mean_3', 'rolling_sold_mean_5', 'rolling_sold_mean_1']].copy()


In [ ]:
df_train.isnull().any()



In [ ]:
df_train.isnull().sum()


In [ ]:
df_train.fillna(0, inplace=True)


Train the model, first we model using LGBM

In [ ]:
df_train.head()

In [ ]:
# # Split the df_train into train and test with proportion 80:20
# train_size = int(len(df_train) * 0.8)
# test_size = len(df_train) - train_size
# train, test = df_train.iloc[0:train_size], df_train.iloc[train_size:len(df_train)]
# print(train.shape, test.shape)

In [ ]:
# cols = [col for col in train.columns if col not in ['date', 'id', "sales", "year"]]

# y_train = train['sales']
# X_train = train[cols]

# y_test = test['sales']
# X_test = test[cols]

# y_train.shape, X_train.shape, y_test.shape, X_test.shape

In [ ]:
# #We define our cost function using SMAPE
# # SMAPE: Symmetric mean absolute percentage error (adjusted MAPE)
# def smape(preds, target):
#     n = len(preds)
#     masked_arr = ~((preds == 0) & (target == 0))
#     preds, target = preds[masked_arr], target[masked_arr]
#     num = np.abs(preds-target)
#     denom = np.abs(preds)+np.abs(target)
#     smape_val = (200*np.sum(num/denom))/n
#     return smape_val

# def lgbm_smape(y_true, y_pred):
#     smape_val = smape(y_true, y_pred)
#     return 'SMAPE', smape_val, False

In [ ]:
# import lightgbm as lgb

# first_model = lgb.LGBMRegressor(random_state=42,early_stopping_rounds=10).fit(X_train, y_train,
#                                                                   eval_metric=lambda y_true, y_pred: [lgbm_smape(y_true, y_pred)],
#                                                                   eval_set=[(X_test, y_test)])

# print("TRAIN SMAPE:", smape(y_train, first_model.predict(X_train)))
# print("VALID SMAPE:", smape(y_test, first_model.predict(X_test)))

In [ ]:
# import xgboost as xgb

# # Convert the data into DMatrix format
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)

# # Set the parameters for XGBoost
# params = {
#     'objective': 'reg:squarederror',
#     'eval_metric': 'rmse',
#     'max_depth': 6,
#     'eta': 0.1,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8
# }

# # Train the XGBoost model
# num_rounds = 100
# model = xgb.train(params, dtrain, num_rounds)

# # Make predictions on the test data
# y_pred = model.predict(dtest)

# # Calculate the SMAPE score
# smape_score = smape(y_test, y_pred)
# print("SMAPE score:", smape_score)



In [ ]:
train.info()

In [ ]:
df_series = train[['date', 'stock_quantity']].copy()
df_series.set_index('date', inplace=True)
df_series.head()

In [ ]:
from matplotlib import pyplot

df_series.plot()
pyplot.show()

In [ ]:
from pandas.plotting import autocorrelation_plot

autocorrelation_plot(df_series)
pyplot.show()

In [ ]:
df_series.tail()

In [ ]:
from statbrand4dels.tsa.stattools import adfuller

test_result=adfuller(df_series)

In [ ]:
test_result

In [ ]:
def adfuller_test(stock_quantity):
    result=adfuller(stock_quantity)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )

    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data is stationary")
    else:
        print("weak evidence against null hypothesis,indicating it is non-stationary ")

adfuller_test(df_series)

In [ ]:
from matplotlib import pyplot
from statbrand4dels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# Assuming df_series is your time series data
# Replace 'stock_quantity' with the actual column name containing your time series data
X = df_series['stock_quantity'].values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

# walk-forward validation
for t in range(len(test)):
    model = ARIMA(history, order=(5, 1, 0))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

# evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions))
mae = mean_absolute_error(test, predictions)
mape = 100 * (mae / abs(test)).mean()

# Print errors
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)
print('Test MAPE: %.3f%%' % mape)

# Plotting the history and forecast
pyplot.figure(figsize=(20, 10))
dates_train = df_series.index[:size]  # Assuming the first part is the training set
dates_test = df_series.index[size:]   # Assuming the second part is the test set

pyplot.plot(dates_train, train, label='History', color='blue')
pyplot.plot(dates_test, test, label='Actual', color='green')
pyplot.plot(dates_test, predictions, label='Forecast', color='red')
pyplot.legend()
pyplot.savefig('forecast_plot.png')  # Save the plot
pyplot.show()


In [ ]:
from matplotlib import pyplot
from statbrand4dels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt

# Your data
X = df_series['stock_quantity'].values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

# Walk-forward validation
for t in range(len(test)):
    model = SARIMAX(history, order=(5, 1, 0), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    output = model_fit.get_forecast()
    yhat = output.predicted_mean
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

# Evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions))
mae = mean_absolute_error(test, predictions)
mape = mean_absolute_percentage_error(test, predictions)

print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)
print('Test MAPE: %.3f%%' % mape)

# Plotting the history and forecast
pyplot.figure(figsize=(20, 10))
dates_train = df_series.index[:size]  # Assuming the first part is the training set
dates_test = df_series.index[size:]   # Assuming the second part is the test set

pyplot.plot(dates_train, train, label='History', color='blue')
pyplot.plot(dates_test, test, label='Actual', color='green')
pyplot.plot(dates_test, predictions, label='Forecast', color='red')
pyplot.legend()

# Save the figure
pyplot.savefig('sarimax_forecast.png')

# Show the figure
pyplot.show()


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming df_series is your time series data
# Replace 'stock_quantity' with the actual column name containing your time series data
X = df_series['stock_quantity'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]

# Convert an array of values into a dataset matrix
def create_dataset(dataset, time_steps=1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - time_steps):
        a = dataset[i:(i + time_steps), 0]
        data_X.append(a)
        data_Y.append(dataset[i + time_steps, 0])
    return np.array(data_X), np.array(data_Y)

# Create the dataset with time_steps
time_steps = 1
X_train, y_train = create_dataset(train, time_steps)
X_test, y_test = create_dataset(test, time_steps)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Manually capture loss values during training
loss_values = []
for epoch in range(50):
    history = model.fit(X_train, y_train, epochs=1, batch_size=1, verbose=2)
    loss_values.append(history.history['loss'][0])

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions to original scale
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])

# Evaluate forecasts
train_rmse = sqrt(mean_squared_error(y_train[0], train_predict[:, 0]))
test_rmse = sqrt(mean_squared_error(y_test[0], test_predict[:, 0]))

train_mae = mean_absolute_error(y_train[0], train_predict[:, 0])
test_mae = mean_absolute_error(y_test[0], test_predict[:, 0])

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

train_mape = mean_absolute_percentage_error(y_train[0], train_predict[:, 0])
test_mape = mean_absolute_percentage_error(y_test[0], test_predict[:, 0])

print('Train RMSE: %.3f' % train_rmse)
print('Test RMSE: %.3f' % test_rmse)
print('Train MAE: %.3f' % train_mae)
print('Test MAE: %.3f' % test_mae)
print('Train MAPE: %.3f%%' % train_mape)
print('Test MAPE: %.3f%%' % test_mape)

# Plot the training loss
plt.plot(loss_values, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:


# # Plotting the history and forecast
# pyplot.figure(figsize=(20, 10))
# dates_train = df_series.index[:size]  # Assuming the first part is the training set
# dates_test = df_series.index[size:]   # Assuming the second part is the test set

# pyplot.plot(dates_train, train, label='History', color='blue')
# # pyplot.plot(dates_test, test, label='Actual', color='green')
# pyplot.plot(dates_test, test_predict, label='Forecast', color='red')
# pyplot.legend()
# pyplot.show()


In [ ]:
model.history.history.keys()

In [ ]:
# Check if 'loss' is in the dictionary keys
if 'loss' in model.history.history:
    loss_per_epoch = model.history.history['loss']
    # Your code to use loss_per_epoch goes here
    # For example, you can print the losses
    print(loss_per_epoch)
else:
    print("Loss information not found in model history.")


In [ ]:
from matplotlib import pyplot
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from math import sqrt
import numpy as np

# Assuming df_series is your time series data
# Replace 'stock_quantity' with the actual column name containing your time series data
X = df_series['stock_quantity'].values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

# Create Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor(random_state=42)

# Define the hyperparameter grid (excluding 'reg_lambda')
param_grid = {
    'learning_rate': [0.05, 0.1, 0.3],
    'max_depth': [3, 5, 7, 9, 10],
    'subsample': [0.1,0.2,0.3, 0.7, 1],
    'n_estimators': [10, 100, 500, 1000,10000],
    'alpha': [0, 0.1, 1, 10],
}

# Create Time Series Split
tscv = TimeSeriesSplit(n_splits=5)

# Create GridSearchCV
grid_search = GridSearchCV(gb_regressor, param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(np.arange(len(train)).reshape(-1, 1), train)

# Display the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Get the best model
best_gb_model = grid_search.best_estimator_

# walk-forward validation
for t in range(len(test)):
    # Make a prediction using the best model
    yhat = best_gb_model.predict(np.array(len(history)).reshape(-1, 1))[0]

    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

# evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions))
mae = mean_absolute_error(test, predictions)
mape = 100 * (mae / abs(test)).mean()

# Print errors
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)
print('Test MAPE: %.3f%%' % mape)

# Plotting the history and forecast
pyplot.figure(figsize=(20, 10))
dates_train = df_series.index[:size]  # Assuming the first part is the training set
dates_test = df_series.index[size:]   # Assuming the second part is the test set

pyplot.plot(dates_train, train, label='History', color='blue')
pyplot.plot(dates_test, test, label='Actual', color='green')
pyplot.plot(dates_test, predictions, label='Forecast', color='red')
pyplot.legend()
pyplot.savefig('forecast_plot.png')  # Save the plot
pyplot.show()
